En este apartado se hicieron todos las tranformaciones necesarias para las funciones que va a consumir la API. Se tienen como fuentes de datos los archivos ya limpios: user_reviews_cleaned, steam_games_cleaned, user_items_cleaned. Se persigue la eficiencia y ligereza de estas consultas para facilitar el trabajo de la API.

### Importar Bibliotecas

In [1]:
import pandas as pd
import os
import ast

### Cargar el conjunto de datos limpios

In [2]:
df_steam_games = pd.read_csv('steam_games_cleaned.csv')
df_user_reviews = pd.read_csv('user_reviews_cleaned.csv')
df_user_items = pd.read_csv('user_items_cleaned.csv')

### Rutinas

In [ ]:
df_reviews = pd.read_csv('data/df_reviews_unido.csv')
df_gastos_items = pd.read_csv('data/df_gastos_items_unido.csv')
df_items_developer = pd.read_csv('data/df_items_developer_unido.csv')

### developer 


Esta función recibe como parámetro 'developer', que es la empresa desarrolladora del juego, y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido Free por año por sobre el total que desarrolla

In [ ]:
def developer(desarrollador):
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [ ]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

### user data 

In [ ]:
def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario = df_user_reviews[df_reviews['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [ ]:
user_id = 'EchoXSilence'
userdata(user_id)

### UserForGenre 

In [ ]:
df_game = df_steam_games.drop(['name', 'developer', 'price', 'tags','specs'],axis=1)

# Merge de los DataFrames
merged_data=pd.merge(df_user_items,df_game,left_on='item_id', right_on='item_id', how='right')

merged_data.drop(['item_id'], axis=1, inplace=True)
merged_data.dropna(inplace=True)

merged_data['hours_game'] = merged_data['hours_game'].astype(int)
merged_data['year'] = merged_data['year'].astype(int)

#Convertimos la columna genres a tipo lista
merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)

#Expandimos las listas en columnas
merged_data = merged_data.explode('genres').reset_index(drop=True)

#Encontrar el jugador con más horas jugadas por género y año
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'year'])['hours_game'].idxmax()]

#Filtrar las columnas necesarias
max_hours_df = max_hours_df[['genres', 'year', 'user_id', 'hours_game']]

#Segunda agrupación para obtener las horas jugadas por año para el usuario seleccionado
max_hours = max_hours_df.groupby(['genres', 'year', 'user_id'])['hours_game'].sum().reset_index()

max_hours.to_csv('UserForGenre.csv', index=False)

##### best_developer_year 

In [1]:
# Unir los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
 
# Renombrar la columna 'year_x' a 'year'
df_filtered = df_filtered.rename(columns={'year_x': 'year'})

# Contar las ocurrencias de cada desarrolladora por año
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
    
# Ordenar por 'year' y 'count' en orden descendente
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    

# Obtener el top 3 por año
result_df = grouped_result.groupby('year').head(3)    

# Guardar el resultado en un archivo CSV
result_df.to_csv('best_developer_year.csv', index=False)

NameError: name 'pd' is not defined

##### developer_review_analysis

In [5]:
# Merge de los DataFrames
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_csv('developer_reviews_analysis.csv', index=False)